In [1]:
#re.findall('\(.*?\)',b1)
#re.findall('\(.*?\)',f1)

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

from goodreads import client
gc = client.GoodreadsClient('9addHIFaPJmit7dzC5ZA', 'iDXvx13Rnt1iJkWOxRkSh9wLnbFWmMNgCUMFLJmLPo')

In [3]:
wikipages = ['https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)?oldformat=true'
            ]

In [4]:
def add_to_rows(page_soup):
    """
    Method extracts row elements from the soup tags of individual 
    list pages (A - F, H - J, etc.) and appends them to the `rows` 
    list.
    
    Args:
        page_soup: (BeautifulSoup tag) soup for list page.
    """
    
    global rows
    
    tables = page_soup.find_all('table', {'class': 'wikitable'})

    for alphabet in tables:
        rows += alphabet.findAll('tr')
    
    print("Tables added: {}, Rows added: {}".format(len(tables), len(rows)))

In [5]:
def working_wiki_link(wiki_link):
    """
    Converts relative URLs to global URLs.
    """
    
    if '/wiki/' in wiki_link:
        return 'https://en.wikipedia.org' + wiki_link
    return wiki_link

In [ ]:
def get_adaptations(book_title, href_list):
    """
    Parses through list of movies associated with a specific `book_title`,
    adds movies to `movie_book_dict`, and returns list of movies.
    Adds year to the movie title if information available. 
    
    Args:
        book_title: (str) name of original book.
        href_list: (list) Result of findAll('a') on the movies column.
        
    Returns:
        adaptations: List of movie title names.
    """
    adaptations = []
    
    if len(href_list) == 1:
        adaptations.append(href_list[0]['title'])
        return adaptations
        
    for index, tag in enumerate(href_list):
        year_search = re.search(r'\d\d\d\d', tag['href'])
        
        if year_search is not None:  #There is a 4 digit number in the title
            year_search = year_search.group()
        
            if year_search not in tag['title']:
                version = tag['title'] + ' (' + year_search + ')'
            else:
                version = tag['title']
            
            adaptations.append(version)
            movie_book_dict[working_wiki_link(tag['href'])] = {'movie_title': version, 
                                                              'book': book_title}
        
        else:
            version = tag['title']
            adaptations.append(version)
            movie_book_dict[working_wiki_link(tag['href'])] = {'movie_title': version, 
                                                              'book': book_title}

    
    return adaptations

def clean_adaptations(adaptations):
    """
    Makes sure the list of adaptations doesn't contain TV series or miniseries.
    """
    
    cleaned = []
    
    for title in adaptations:
        if 'TV' in title or 'miniseries' in title:
            continue
        if 'page does not exist' in title:
            continue
        else:
            cleaned.append(title)
            
    return cleaned

In [14]:
def add_to_book_film_dict():
    """
    Method parses newly updated rows for book/film title, book author,
    number of adaptions, and list of adaptations, and adds them to the
    `book_film_dict` as a nested dictionary. 
    global sum_adaptations
    """
    global sum_adaptations
    
    for row in rows:
        cols = row.findAll('td')

        try:
            if (len(cols[0].findAll('a')) == 2) and (cols[1].find('a') is not None):
                series = cols[0].text.replace('\n', '').split(',')[0]

                book_title = cols[0].findAll('a')[0]['title']
                author = cols[0].findAll('a')[-1]['title']

                book_wiki_url = working_wiki_link(cols[0].findAll('a')[0]['href'])

                try:
                    adaptations = clean_adaptations(get_adaptations(book_title, cols[1].findAll('a')))
                except KeyError:
                    adaptations = []
                    pass

                sum_adaptations += len(adaptations)

                book_film_dict[series] = {'author': author,
                                            'book_title': book_title,
                                            'book_wiki_url': book_wiki_url,
                                            'count': len(adaptations),
                                            'adaptations': adaptations,
                                            'isbn': '', 'oclc':'', 'valid_identifier': ''}

        except IndexError or KeyError:
            pass
        
    print("No. of rows/original books in dict: {}, No. of adaptations: {} \n"
          .format(len(book_film_dict), sum_adaptations))

In [15]:
rows = []
book_film_dict = {}
movie_book_dict = {}
sum_adaptations = 0

for url in wikipages:
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    
    add_to_rows(soup)
    add_to_book_film_dict()

Tables added: 25, Rows added: 376
No. of rows/original books in dict: 259, No. of adaptations: 754 

Tables added: 3, Rows added: 798
No. of rows/original books in dict: 553, No. of adaptations: 2046 

Tables added: 7, Rows added: 1245
No. of rows/original books in dict: 813, No. of adaptations: 3917 

Tables added: 8, Rows added: 1662
No. of rows/original books in dict: 1056, No. of adaptations: 6426 

Tables added: 7, Rows added: 2034
No. of rows/original books in dict: 1237, No. of adaptations: 9466 



In [16]:
len_initial = len(rows)
len_initial

2034

In [19]:
movie_book_dict['https://en.wikipedia.org/wiki/102_Dalmatians'], movie_book_dict['https://en.wikipedia.org/wiki/3_Godfathers']

({'movie_title': '102 Dalmatians', 'book': 'The Hundred and One Dalmatians'},
 {'movie_title': '3 Godfathers', 'book': 'Three Godfathers'})

## Don't touch above

In [ ]:
def get_isbn(wiki_url):
    """
    Parses the `infobox vcard` element of the wikipedia page of a book, 
    and returns the 10-digit or 13-digit ISBN number (if found). 
    
    Args:
        wiki_url (str): Valid (not local) wikipedia link.

    Returns:
        isbn: Returns ISBN code is parsed correctly, 'broken' if formmated 
        incorrectly or otherwise. The code is de-hyphentated for later pipeline
        efficiency. 
 
    """
    try:
        page = requests.get(wiki_url).text
        soup = BeautifulSoup(page, 'lxml')
    except MissingSchema:
        return 'broken'  
    
    infobox = soup.find('table', {'class': 'infobox vcard'})
    
    if infobox is None:
        return 'broken'
    
    for row in infobox.findAll('tr'):
        #ISBN directly available
        if ('ISBN' in row.text):
            isbn = re.sub('[^0-9]','', row.findAll('a')[-1].text)
            return isbn
        
    #Nothing found directly on Infobox
    return 'broken' 

In [ ]:
# Only to be used if get_isbn() returns 'not formatted correctly'
def get_oclc(wiki_url):
    """
    Parses the `infobox vcard` element of the wikipedia page of a book, 
    and returns the 10-digit or 13-digit OCLC number (if found). This function is a backup
    if get_isbn() returns 'broken', and must not be relied on. 
    
    Args:
        wiki_url (str): Valid (not local) wikipedia link.

    Returns:
        oclc: Returns OCLC code is parsed correctly, 'broken' if formmated 
        incorrectly or otherwise. 
 
    """
    
    try:
        page = requests.get(wiki_url).text
        soup = BeautifulSoup(page, 'lxml')
    except MissingSchema:
        return 'broken'

    infobox = soup.find('table', {'class': 'infobox vcard'})
    
    if infobox is None:
        return 'broken'
    
    for row in infobox.findAll('tr'):
        #only OCLC available
        if row.find('a', {'title': 'OCLC'}) is not None:
            return (row.find('td').text)
        
    #Nothing found directly on Infobox
    return 'broken'     

In [ ]:
broken_count = 0
usable_count = 0
index = 0

for key in book_film_dict.keys():
    
    link = book_film_dict[key]['book_wiki_url']
    
    try:
        book_film_dict[key]['isbn'] = get_isbn(link)
    
        if book_film_dict[key]['isbn'] == 'broken':
            book_film_dict[key]['oclc'] = get_oclc(link)
            if book_film_dict[key]['oclc'] == 'broken':
                book_film_dict[key]['valid_identifer'] = False
                #Nothing changes, the book is unusable, and the 'usable' flag remains False
        else:
            book_film_dict[key]['valid_identifer'] = True
            usable_count += 1
    except NameError:
        pass
        
    if (index % 50 == 0):
        broken_count = index - usable_count
        print("Usable: {}, Broken: {}, Total: {}".format(usable_count, broken_count, index))
    
    index += 1

Usable: 0, Broken: 0, Total: 0
Usable: 25, Broken: 25, Total: 50
Usable: 52, Broken: 48, Total: 100
Usable: 76, Broken: 74, Total: 150


we lose a bunch of old books, but it's okay since inflation would have destroyed old movie sales as well. i guess it wokrs out

## Don't touch above

In [ ]:
book_film_dict['Dreamcatcher (2001)']

In [ ]:
book_film_dict["Gulliver's Travels (1726)"]

In [ ]:
desired_metadata = ['average_rating', 
                    'publication_date',
                    'rating_dist',
                    'ratings_count', 'format',
                    'text_reviews_count',
                    'title', 'language_code']

def get_goodreads_data(gc_code):
    """
    Add additional review and ratings related meta_data from the GoodReads API.
    
    Args:
        gc_code: Strictly Goodreads code. Can be 10-digit or 13-digit.
        
    Returns:
        gr_metadata: Dict with additional metadata.
    """
    
    gr_metadata = {}
    try:
        book = gc.book(gc_code)
    except NameError:
        return gr_metadata
    except ExpatError:
        return gr_metadata
    
    for attrs in book.__dict__['_book_dict']:
        if attrs in desired_metadata:
            gr_metadata['gc_' + attrs] = book.__dict__['_book_dict'][attrs]
            
    return gr_metadata

In [ ]:
meta_data_wanted = ['identifiers', 'number_of_pages', 'subject_places', 'subjects', 'publish_date', 'publish_places']

def update_book_metadata(book_title):
    """
    Uses the OpenLibrary and Goodreads APIs (when possible) to 
    add metadata about the books to the `book_film_dict`.
    
    Args: 
        book_title: (str) used when merging back into `book_film_dict`.
    
    Returns:
        updated: (bool) True if metadata updated, False otherwise.
    
    
    """
    metadata = {}
    all_metadata = {}
    
    
    if book_film_dict[book_title]['valid_identifer'] is False:
        #API can't be accessed
        metadata['metadata_updated'] = False
        book_film_dict[book_title].update(metadata)
        return
    
    elif book_film_dict[book_title]['isbn'] != '':
        code = book_film_dict[book_title]['isbn']
        curl = 'https://openlibrary.org/api/books?bibkeys=ISBN:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
    else:
        code = book_film_dict[book_title]['oclc']
        curl = 'https://openlibrary.org/api/books?bibkeys=OCLC:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
        
    if len(all_metadata) > 0:
        key = list(all_metadata.keys())[0]
        nested_keys = list(all_metadata[key].keys())
        
        for data_point in meta_data_wanted:
            if data_point in nested_keys:
                metadata[data_point] = all_metadata[key][data_point]
    
        if 'subject_places' in nested_keys:
            metadata['subject_places'] = [place['name'][:30] for place in metadata['subject_places']]
        if 'subjects' in nested_keys:
            metadata['subjects'] = [subject['name'][:30] for subject in metadata['subjects']]
        if 'publish_places' in nested_keys:
            metadata['publish_places'] = [place['name'][:30] for place in metadata['publish_places']]            
        
        if 'identifiers' in nested_keys:
            if 'goodreads' in list(metadata['identifiers'].keys()):
                metadata['goodreads'] = metadata['identifiers']['goodreads'][0]
            metadata.pop('identifiers')
            
        if 'goodreads' in metadata.keys():
            metadata.update(get_goodreads_data(metadata['goodreads']))
            
        metadata['metadata_updated'] = True
        book_film_dict[book_title].update(metadata)
        return
    
    else:
        metadata['metadata_updated'] = False
        book_film_dict[book_title].update(metadata)
        return


In [ ]:
update_book_metadata('Dreamcatcher (2001)')
book_film_dict['Dreamcatcher (2001)']

don't run below till the end

In [ ]:
from xml.parsers.expat import ExpatError

metadata_update_count = 0
metadate_update_dict = {}

for index, key in enumerate(book_film_dict.keys()):
    
    update_book_metadata(key)
    
    if book_film_dict[key]['metadata_updated'] is True:
        metadata_update_count += 1
        
    metadate_update_dict[index] = metadata_update_count
    
    if index % 75 == 0 and index != 0:
        print("Indexed: {}, Updated: {}, Usable: {}%"
              .format(index, metadata_update_count, round(100*metadata_update_count/index, 3)))

In [ ]:
print("Done. Started with {} rows; now {} are usable: ~{}%."
    .format(len(book_film_dict), metadata_update_count,round(100*metadata_update_count/len(book_film_dict), 3)))

## Bringing in to pandas

In [ ]:
book_film_dict

In [ ]:
book_film = pd.DataFrame()
rows = []

for index, key in enumerate(book_film_dict.keys()):
    get_book_metadata(key)
    
    for movie in book_film_dict[key]['adaptations']:
        row = [key, book_film_dict[key]['author'], 
               movie, 
               book_film_dict[key]['count'], 
               book_film_dict[key]['isbn'],
               book_film_dict[key]['oclc'],
               book_film_dict[key]['usable'],
        rows.append(row)
        
    if index > 20:
        break
    
book_film = pd.DataFrame(data=rows)
book_film.rename(columns={0: 'book_title', 
                          1: 'author', 
                          2: 'movie', 
                          3: 'total_ad_count', 
                          4: 'isbn', 5: 'oclc', 
                          6: 'usable'},
                inplace = True)

book_film.head()